In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from collections import Counter

In [2]:
df_item = pd.read_csv(r'C:\PycharmProjects\PythonProject\recommendation_for_films\df_items_final.csv')
df_item.head()

,item_id,name,description,genres,is_generated,media_type,release_year,decade,rating_scaled
0,m_27205,Inception,"Cobb, a skilled thief who commits corporate es...","Action, Science Fiction, Adventure",0,movie,2010.0,2010.0,4.26380
1,m_157336,Interstellar,The adventures of a group of explorers who mak...,"Adventure, Drama, Science Fiction",0,movie,2014.0,2010.0,4.28765
2,m_155,The Dark Knight,Batman raises the stakes in his war on crime. ...,"Drama, Action, Crime, Thriller",0,movie,2008.0,2000.0,4.33040
3,m_19995,Avatar,"In the 22nd century, a paraplegic Marine is di...","Action, Adventure, Fantasy, Science Fiction",0,movie,2009.0,2000.0,3.90785
4,m_24428,The Avengers,When an unexpected enemy emerges and threatens...,"Science Fiction, Action, Adventure",0,movie,2012.0,2010.0,3.96950


In [3]:
df_item["genres"] = df_item["genres"].str.split(", ")
all_genres = [g.strip() for sub in df_item["genres"].dropna() for g in sub]

In [4]:
genre_counts = Counter(all_genres)
genre_df = pd.DataFrame(genre_counts.items(),columns=["genre", "count"]).sort_values("count", ascending=False)
print("Всего уникальных жанров:", len(genre_df))
print(genre_df.head(20))

Всего уникальных жанров: 29
                 genre   count
3                Drama  142534
19             Unknown  122388
7               Comedy   98554
18         Documentary   69616
12           Animation   33172
8              Romance   31168
5             Thriller   28846
0               Action   26933
4                Crime   26833
14              Horror   25231
13              Family   21905
15               Music   19270
10             Mystery   16578
2            Adventure   14645
17            TV Movie   14237
1      Science Fiction   12086
6              Fantasy   11942
25             Reality   10755
16             History    8568
21  Action & Adventure    7196


In [5]:
mask = df_item["genres"].astype(str).str.contains("Unknown", na=False)
unknown_df = df_item[mask][["item_id", "name", "description", "media_type", "release_year", "genres"]]
unknown_df.head(10)

,item_id,name,description,media_type,release_year,genres
6011,m_320367,Return,A tale of terror. Cathy Reed has been institut...,movie,2015.0,[Unknown]
7685,m_916035,Barely Legal Baby Fat,Nobody finds hotter girls just days after the ...,movie,2008.0,[Unknown]
8576,m_50472,Anplagghed al cinema,"A queue at the ATM machine, a displaced family...",movie,2006.0,[Unknown]
9795,m_464446,Return,A young man returns home for the weekend to di...,movie,2015.0,[Unknown]
10567,m_292141,The Right of Youth,Directed by August Blom.,movie,1911.0,[Unknown]
11158,m_1086372,Return,The main character of the film is an outstandi...,movie,1972.0,[Unknown]
11404,m_596601,Prison High Pressure,Behind the scenes in a prison with a very spec...,movie,2019.0,[Unknown]
11631,m_834860,RETURN,8mm work directed by Norihiko Morinaga.,movie,1985.0,[Unknown]
13026,m_83050,Hot Chocolate: A Romp in the Dark,Hot Chocolate is a study in the perfection tha...,movie,2008.0,[Unknown]
14391,m_348877,Return,Static images of an old country house are comb...,movie,1972.0,[Unknown]


In [6]:
no_genre_mask = (
    df_item["genres"].isna() |
    df_item["genres"].astype(str).str.strip().eq("") |
    df_item["genres"].astype(str).str.contains("Unknown", na=False))
unknown_df = df_item[no_genre_mask][["item_id", "name", "media_type", "release_year", "genres"]]
unknown_df.head(10)

,item_id,name,media_type,release_year,genres
6011,m_320367,Return,movie,2015.0,[Unknown]
7685,m_916035,Barely Legal Baby Fat,movie,2008.0,[Unknown]
8576,m_50472,Anplagghed al cinema,movie,2006.0,[Unknown]
9795,m_464446,Return,movie,2015.0,[Unknown]
10567,m_292141,The Right of Youth,movie,1911.0,[Unknown]
11158,m_1086372,Return,movie,1972.0,[Unknown]
11404,m_596601,Prison High Pressure,movie,2019.0,[Unknown]
11631,m_834860,RETURN,movie,1985.0,[Unknown]
13026,m_83050,Hot Chocolate: A Romp in the Dark,movie,2008.0,[Unknown]
14391,m_348877,Return,movie,1972.0,[Unknown]


In [7]:
media_dist = df_item["media_type"].value_counts(normalize=True)
media_dist

media_type
movie    0.676827
tv       0.323173
Name: proportion, dtype: float64

In [8]:
year_bins = pd.cut(df_item["release_year"].dropna(),
                   bins=[1980,1990,2000,2010,2020,2025],
                   labels=["1980-89","1990-99","2000-09","2010-19","2020-25"])
print(year_bins.value_counts(normalize=True))

release_year
2010-19    0.434194
2000-09    0.219885
2020-25    0.165879
1990-99    0.104007
1980-89    0.076036
Name: proportion, dtype: float64


In [9]:
df_unique = df_item.drop_duplicates(subset=['item_id'], keep='first').copy()
print(f"Размер датасета после удаления дубликатов по item_id: {df_unique.shape}")

Размер датасета после удаления дубликатов по item_id: (509628, 9)


In [10]:
initial_shape = df_unique.shape
df_unique_clean_years = df_unique.dropna(subset=['release_year']).copy()
df_unique_clean_years.reset_index(drop=True, inplace=True)
final_shape = df_unique_clean_years.shape

print(f"Размер датасета после удаления строк с пропущенным 'release_year': {final_shape}")
print(f"Удалено строк с пропущенным 'release_year': {initial_shape[0] - final_shape[0]}")

Размер датасета после удаления строк с пропущенным 'release_year': (478727, 9)
Удалено строк с пропущенным 'release_year': 30901


In [11]:
remaining_missing = df_unique_clean_years.isnull().sum()
if remaining_missing['release_year'] == 0:
    print("Проверка: Пропущенные значения в 'release_year' успешно удалены.")
else:
    print(f"ВНИМАНИЕ: Остались пропущенные 'release_year': {remaining_missing['release_year']}")

Проверка: Пропущенные значения в 'release_year' успешно удалены.


In [12]:
unknown_mask = (
    df_unique_clean_years["genres"].isna() |
    df_unique_clean_years["genres"].astype(str).str.strip().eq("") |
    df_unique_clean_years["genres"].astype(str).str.contains("Unknown", case=False, na=False)
)

In [13]:
df_clean = df_unique_clean_years[~unknown_mask].copy()
df_unknown = df_unique_clean_years[unknown_mask].copy()
print(f"Количество записей с известными жанрами: {len(df_clean)}")
print(f"Количество записей с неизвестными жанрами: {len(df_unknown)}")

Количество записей с известными жанрами: 380391
Количество записей с неизвестными жанрами: 98336


In [14]:
TARGET_TOTAL = 20_000
UNKNOWN_PERCENTAGE = 0.04 # 4%
N_UNKNOWN = int(TARGET_TOTAL * UNKNOWN_PERCENTAGE)
N_CLEAN = TARGET_TOTAL - N_UNKNOWN
print(f"Будет выбрано: {N_CLEAN} записей с известными жанрами и {N_UNKNOWN} с неизвестными.")

Будет выбрано: 19200 записей с известными жанрами и 800 с неизвестными.


In [15]:
bins = [1980, 1990, 2000, 2010, 2020, 2026]
labels = ["1980-89", "1990-99", "2000-09", "2010-19", "2020-25"]
df_clean["year_bin"] = pd.cut(df_clean["release_year"], bins=bins, labels=labels, right=False)

In [16]:
strata = df_clean.groupby(["media_type", "year_bin"], observed=False, dropna=False)

In [17]:
strata_weights = strata.size() / len(df_clean)
print("\nВеса страт:")
print(strata_weights)


Веса страт:
media_type  year_bin
movie       1980-89     0.053724
            1990-99     0.064849
            2000-09     0.124107
            2010-19     0.244719
            2020-25     0.109043
            nan         0.165319
tv          1980-89     0.010071
            1990-99     0.017663
            2000-09     0.038926
            2010-19     0.091603
            2020-25     0.067512
            nan         0.012463
dtype: float64


In [18]:
strata_sample_counts = (strata_weights * N_CLEAN).round().astype(int)

In [19]:
diff = N_CLEAN - strata_sample_counts.sum()
if diff != 0:
    # Найти страту с наибольшим количеством записей и скорректировать её
    # (можно использовать более сложную логику, но это простой способ)
    largest_stratum = strata_sample_counts.idxmax()
    strata_sample_counts[largest_stratum] += diff
    print(f"\nКорректировка количества выборки: добавлено {diff} к страте {largest_stratum}")

print(f"\nЦелевое количество выборки по стратам:")
print(strata_sample_counts)
print(f"Сумма: {strata_sample_counts.sum()}")


Корректировка количества выборки: добавлено 1 к страте ('movie', '2010-19')

Целевое количество выборки по стратам:
media_type  year_bin
movie       1980-89     1031
            1990-99     1245
            2000-09     2383
            2010-19     4700
            2020-25     2094
            nan         3174
tv          1980-89      193
            1990-99      339
            2000-09      747
            2010-19     1759
            2020-25     1296
            nan          239
dtype: int64
Сумма: 19200


In [20]:
def safe_sample(group, n):
    """Безопасная выборка: если в группе меньше элементов, чем n, выбираем все."""
    n = min(len(group), n)
    if n <= 0:
         return group.iloc[0:0] # Возвращаем пустой DataFrame с теми же колонками
    return group.sample(n=n, random_state=42)

In [21]:
sampled_clean_list = []
for name, group in strata:
    count = strata_sample_counts.get(name, 0) # Получаем целевое количество, 0 если ключ отсутствует
    if count > 0 and not group.empty: # Добавлена проверка на пустую группу
        sampled_group = safe_sample(group, count)
        sampled_clean_list.append(sampled_group)
    elif count > 0 and group.empty:
        print(f"Предупреждение: Для страты {name} целевое количество выборки {count}, но группа пуста.")

In [22]:
if sampled_clean_list:
    sampled_clean_final = pd.concat(sampled_clean_list, ignore_index=True)
else:
    sampled_clean_final = pd.DataFrame() # На случай, если все страты дали 0

print(f"\nРеально выбрано из 'чистых' данных: {len(sampled_clean_final)}")


Реально выбрано из 'чистых' данных: 19200


In [23]:
if len(df_unknown) >= N_UNKNOWN:
    sampled_unknown_final = df_unknown.sample(n=N_UNKNOWN, random_state=42)
    print(f"Выбрано из 'unknown' данных: {len(sampled_unknown_final)}")
else:
    print(f"Предупреждение: Недостаточно записей с 'Unknown' жанрами. Доступно: {len(df_unknown)}, требуется: {N_UNKNOWN}")
    print("Будут выбраны все доступные записи с 'Unknown' жанрами.")
    sampled_unknown_final = df_unknown.copy()
    # Корректируем итоговое количество, если недобор по Unknown
    actual_unknown_selected = len(sampled_unknown_final)
    actual_total = len(sampled_clean_final) + actual_unknown_selected
    print(f"Фактический размер выборки будет: {actual_total}")

Выбрано из 'unknown' данных: 800


In [24]:
sampled_final = pd.concat([sampled_clean_final, sampled_unknown_final], ignore_index=True)

In [25]:
sampled_final = sampled_final.sample(frac=1, random_state=42).reset_index(drop=True)

In [26]:
required_columns = ['item_id', 'name', 'genres', 'description', 'release_year', 'decade', 'media_type', 'rating_scaled', 'is_generated']
missing_cols = set(required_columns) - set(sampled_final.columns)

if missing_cols:
    print(f"Warning: Missing columns in final dataset: {missing_cols}")
else:
    sampled_final = sampled_final[required_columns]

In [27]:
print(f"\nФинальный размер выборки: {len(sampled_final)}")
print("\nРаспределение media_type в выборке:")
print(sampled_final['media_type'].value_counts(normalize=True))


Финальный размер выборки: 20000

Распределение media_type в выборке:
media_type
movie    0.75305
tv       0.24695
Name: proportion, dtype: float64


In [28]:
selected_unknown_item_ids = set(sampled_unknown_final['item_id'])
final_unknown_mask = sampled_final['item_id'].isin(selected_unknown_item_ids)
final_unknown_count = final_unknown_mask.sum() # sum() считает True как 1, False как 0
final_unknown_percentage = final_unknown_count / len(sampled_final) * 100
print(f"\nКоличество записей с 'Unknown' жанрами в финальной выборке: {final_unknown_count} ({final_unknown_percentage:.2f}%)")


Количество записей с 'Unknown' жанрами в финальной выборке: 800 (4.00%)


In [29]:
if 'year_bin' in sampled_final.columns:
     print("\nРаспределение годовых бинов в 'чистой' части выборки:")
     # Для чистой части, так как у unknown нет year_bin
     clean_part = sampled_final[sampled_final.index.isin(sampled_clean_final.index)]
     print(clean_part['year_bin'].value_counts(normalize=True, sort=False))

In [30]:
required_columns = ['item_id', 'name', 'description', 'genres', 'is_generated', 'media_type', 'release_year', 'decade', 'rating_scaled']
missing_cols = set(required_columns) - set(sampled_final.columns)
if missing_cols:
    print(f"Внимание: Следующие колонки отсутствуют в финальном датасете: {missing_cols}")
else:
    # Выбираем только нужные колонки
    final_dataset = sampled_final[required_columns].copy()
    output_path = "llama3_20k_balanced.csv"
    final_dataset.to_csv(output_path, index=False)
    print(f"\nФинальный датасет сохранен в: {output_path}")
    print(f"Колонки в сохраненном файле: {list(final_dataset.columns)}")


Финальный датасет сохранен в: llama3_20k_balanced.csv
Колонки в сохраненном файле: ['item_id', 'name', 'description', 'genres', 'is_generated', 'media_type', 'release_year', 'decade', 'rating_scaled']


In [31]:
sampled_final.head()

,item_id,name,genres,description,release_year,decade,media_type,rating_scaled,is_generated
0,m_1373727,Caligula: The Ultimate Cut,"[History, Drama]",Follows Caligula as he kills his devious adopt...,2023.0,2020.0,movie,4.32500,0
1,m_77624,Mesmer,[Drama],A biography of the eighteenth century Viennese...,1994.0,1990.0,movie,3.24500,0
2,m_570224,Oblomov,[Drama],Unknown,2017.0,2010.0,movie,3.20000,0
3,m_125149,Hell Mountain,"[Thriller, Science Fiction, Action]","In an apocalyptical future, the world is compl...",1998.0,1990.0,movie,1.99985,0
4,m_649877,Adiós Buenos Aires,[Music],A freakish coincidence brings together a songw...,1938.0,1930.0,movie,3.51500,0


In [32]:
sampled_final.shape

(20000, 9)